In [12]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown

In [13]:
base_df = pd.read_csv('base_df.csv')
base_df.tail()

,author,from,index,journal,paper,year
28445,태석준,한국파생상품학회,22432.0,” ｢한국재무관리학회｣ 14 3,한국 주가지수선물시장에서의 차익거래에 관한 연구,1997.0
28446,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0
28447,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0
28448,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0
28449,이승환,한국파생상품학회,22612.0,” 「국제경제연구」 | 15 1,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정,2009.0


In [14]:
base_df['new_paper'] = \
[base_df['paper'][idx] + '////' + str(base_df['year'][idx]).split(".")[0] for idx in range(len(base_df))]

base_df['new_paper'] = base_df['new_paper'].apply(lambda x : x.lower())

base_df.tail()

,author,from,index,journal,paper,year,new_paper
28445,태석준,한국파생상품학회,22432.0,” ｢한국재무관리학회｣ 14 3,한국 주가지수선물시장에서의 차익거래에 관한 연구,1997.0,한국 주가지수선물시장에서의 차익거래에 관한 연구////1997
28446,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996
28447,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996
28448,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996
28449,이승환,한국파생상품학회,22612.0,” 「국제경제연구」 | 15 1,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정,2009.0,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정////2009


In [21]:
paper_ls = [i.lower() for i in base_df['new_paper'].tolist()]
counter_df = pd.DataFrame.from_dict(Counter(paper_ls),orient='index',columns=['count']).sort_values('count',ascending=False)
catch_paper_ls = counter_df[counter_df['count'] > 3].index.tolist()
display(counter_df.head())
len(catch_paper_ls)

,count
‘////nan,280
common risk factors in the returns on stocks and bonds////1993,180
results////2007,114
on persistence in mutual fund performance////1997,108
the pricing of options and corporate liabilities////1973,101


1187

In [47]:
catch_paper_ls = catch_paper_ls[1:] # nan 값은 제외한다.

In [48]:
catch_paper_ls[:10]

['results////2007',
 'on persistence in mutual fund performance////1997',
 'the pricing of options and corporate liabilities////1973',
 'risk return and equilibrium: empirical tests////1973',
 'theory of the firm: managerial behavior agency costs and ownership structure////1976',
 'the cross-section of expected stock returns////1992',
 'corporate financing and investment decisions when firms have information that investors do not have////1984',
 'common risk factors in the returns on bonds and stocks////1993',
 'stock index futures////2006',
 'tunneling////2000']

- 딕셔너리로 만들어놓은 base_df에서 데이터의 산발성이 보인다.
- 따라서 catch_paper_ls 라는 딕셔너리의 딕셔너리를 구축한다. 
- 이는, base_df에서 논문의 이름과 년도를 기준으로 count를 sorting 했을 때, 3번 이상 나온 논문들만 따로 모든 것이다.
- 하지만 위에서 보듯, 
    - `'common risk factors in the returns on stocks and bonds////1993'`
    - `'common risk factors in the returns on bonds and stocks////1993'`
    - 와 같은, uniqueness 가 보장되지 않는 것을 알 수 있다. 
    - 이에 따라서, dictionary of dictionary의 uniqueness 를 보장하기 위한 작업을 아래에서 시행한다.

In [49]:
catch_paper_ls_copy = catch_paper_ls.copy()
tuned_catch_paper_ls = []
excepted_ls = []
similar_match_ls = []

for i in range(5) : 
    print(i)
    for idx1,val1 in enumerate(catch_paper_ls_copy) : 
        for idx2,val2 in enumerate(catch_paper_ls_copy) :
            if idx1 != idx2 : 
                v1 = set(val1.split(' '))
                v2 = set(val2.split(' '))
                try : 
                    if len(set(v1).intersection(v2)) / ((len(v1) + len(v2))/2) > 0.75 : 
                        similar_match_ls.append((val1,val2))
                        if len(v1) > len(v2) :
                            catch_paper_ls_copy.pop(catch_paper_ls_copy.index(val2))
                            excepted_ls.append((val2))
                        else :
                            catch_paper_ls_copy.pop(catch_paper_ls_copy.index(val1))
                            excepted_ls.append(((val1)))                    
                except : pass

len(catch_paper_ls) , len(catch_paper_ls_copy) , len(excepted_ls)

0
1
2
3
4


(1185, 1065, 120)

- 61개가 줄었다. 이를 기반으로 딕셔너리를 생성한다.
- ready_to_taget_dict라는 변수를 생성한다. 이는, 472개로 된 key를 가지고 있고, 이와 유사한 논문 이름들을 자기 자신을 포함해서 리스트 형태의 value를 가진다.

In [50]:
ready_to_target_dict = dict(zip(catch_paper_ls_copy,[[] for i in range(len(catch_paper_ls_copy))]))

In [51]:
append = 0
naive_append = 0

for idx1,val1 in enumerate(base_df['new_paper'].tolist()) : 
    if idx1 % 4000 == 0 : print(idx1,end='\r')
    for idx2,val2 in enumerate(ready_to_target_dict.keys()) :
        
        paper_name = val1.split('////')[0].lower()
        paper_year = val1.split('////')[1].lower()
        target_name = val2.split('////')[0].lower()
        target_year = val2.split('////')[1].lower()
        
        if paper_year != 'nan' :
            if paper_year == target_year :
                paper_name_to_word = set(paper_name.split(' '))
                target_name_to_word = set(target_name.split(' '))
                if len(paper_name_to_word.intersection(target_name_to_word)) / len(paper_name_to_word) > 0.75 : 
                    ready_to_target_dict[val2].append(val1)
                    append += 1
        else : pass
#             paper_name_to_word = set(paper_name.split(' '))
#             target_name_to_word = set(target_name.split(' '))
#             if len(paper_name_to_word.intersection(target_name_to_word)) / len(paper_name_to_word) > 0.9 : 
#                 ready_to_target_dict[val2].append(val1)
#                 naive_append += 1

In [52]:
final_ls = []
process_ls = []
error_ls = []

for idx1 , val in enumerate(base_df['new_paper']) : 
    process_ls = []
    for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
        if val in ready_to_target_dict.get(key_) : 
            process_ls.append(key_)

    if len(process_ls) == 1 :
        final_ls.append(process_ls[0])
    elif len(process_ls) == 0 :
        final_ls.append(val)
    else : final_ls.append(val)
        
len(final_ls) == len(base_df)

True

In [53]:
len(set(final_ls)) , len(set(base_df['new_paper']))

(15086, 15786)

In [54]:
base_df['tuned_new_paper'] = final_ls
base_df.tail()

,author,from,index,journal,paper,year,new_paper,tuned_new_paper
28445,태석준,한국파생상품학회,22432.0,” ｢한국재무관리학회｣ 14 3,한국 주가지수선물시장에서의 차익거래에 관한 연구,1997.0,한국 주가지수선물시장에서의 차익거래에 관한 연구////1997,한국 주가지수선물시장에서의 차익거래에 관한 연구////1997
28446,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996,a yield factor model of interest rates////1996
28447,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996,a yield factor model of interest rates////1996
28448,Duffee D. and Kan R.,한국파생상품학회,22502.0,” Mathematical of Finance | 6,A Yield Factor Model of Interest Rates,1996.0,a yield factor model of interest rates////1996,a yield factor model of interest rates////1996
28449,이승환,한국파생상품학회,22612.0,” 「국제경제연구」 | 15 1,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정,2009.0,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정////2009,통화옵션을 이용한 미래 원/달러 환율의 위험중립 확률분포 추정////2009


In [55]:
from IPython.display import Markdown

In [56]:
display(Markdown('### 이전의 방법론'))
display(pd.DataFrame.from_dict(Counter(base_df['new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))
display(Markdown('### 개선된 방법론'))
display(pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False).head(10))

### 이전의 방법론

,count
‘////nan,280
common risk factors in the returns on stocks and bonds////1993,180
results////2007,114
on persistence in mutual fund performance////1997,108
the pricing of options and corporate liabilities////1973,101
risk return and equilibrium: empirical tests////1973,95
theory of the firm: managerial behavior agency costs and ownership structure////1976,88
the cross-section of expected stock returns////1992,67
corporate financing and investment decisions when firms have information that investors do not have////1984,65
common risk factors in the returns on bonds and stocks////1993,64


### 개선된 방법론

,count
common risk factors in the returns [provider:earticle] on stocks and bonds////1993,306
‘////nan,280
theory of the firm: managerial behavior agency costs and ownership structure////1976,185
on persistence in mutual fund performance////1997,138
risk return and equilibrium : empirical tests////1973,118
results////2007,114
returns to buying winners and selling losers: implications for stock market efficiency////1993,108
the pricing of options and corporate liabilities////1973,106
agency costs of free cash flow : corporate finance and takeovers////1986,100
corporate financing and investment decisions when firms have information that investors do not have////1984,94


In [57]:
paper_counter_df = pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper']),orient='index',columns=['count']).sort_values('count',ascending=False)
paper_counter_df.reset_index(inplace=True)

In [58]:
paper_counter_df.to_csv('paper_counter_df.csv',index=False)

In [59]:
import pickle

In [60]:
pickle_out = open("paper_ready_to_target_dict.pickle","wb")
pickle.dump(ready_to_target_dict, pickle_out)
pickle_out.close()

___________________________

#### 여태까지 저널에 관계없이 전체 논문에 대한 dictionary 와 paper_frequency_table 을 형성하였다. 
#### 아래에서부터는 각 저널별로 paper_frequency_table 을 생성한다. dictionary는 변하지 않는다.

In [61]:
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')

In [62]:
def make_new_paper(base_df) :
    base_df['new_paper'] = \
    [base_df['paper'][idx] + '////' + str(base_df['year'][idx]).split(".")[0] for idx in range(len(base_df))]

    base_df['new_paper'] = base_df['new_paper'].apply(lambda x : x.lower())

    return base_df.loc[:,['author','journal','paper','year','from','new_paper']]

In [63]:
fm_base_df = make_new_paper(fm_base_df)
f_base_df = make_new_paper(f_base_df)
sc_base_df = make_new_paper(sc_base_df)
dr_base_df = make_new_paper(dr_base_df)

In [64]:
ready_to_target_dict = []
with (open("paper_ready_to_target_dict.pickle", "rb")) as openfile:
    while True:
        try:
            ready_to_target_dict.append(pickle.load(openfile))
        except EOFError:
            break

In [65]:
ready_to_target_dict = ready_to_target_dict[0]
len(ready_to_target_dict.keys())

1065

In [66]:
def final_process(base_df,ready_to_target_dict):
    final_ls = []
    process_ls = []
    error_ls = []

    for idx1 , val in enumerate(base_df['new_paper']) : 
        process_ls = []
        for idx2, key_ in enumerate(list(ready_to_target_dict.keys())) : 
            if val in ready_to_target_dict.get(key_) : 
                process_ls.append(key_)

        if len(process_ls) == 1 :
            final_ls.append(process_ls[0])
        elif len(process_ls) == 0 :
            final_ls.append(val)
        else : final_ls.append(val)

    if len(final_ls) == len(base_df) :
        print('Lower uniqueness Better Performance, orgin uniqueness: {} , revised uniqueness : {}'.format(len(set(base_df['new_paper'])),len(set(final_ls))))
        base_df['tuned_new_paper'] = final_ls
        return base_df
    else : raise TypeError('error')

In [67]:
fm_final_df = final_process(fm_base_df,ready_to_target_dict)
fm_final_df = final_process(fm_base_df,ready_to_target_dict)
f_final_df = final_process(f_base_df,ready_to_target_dict)
sc_final_df = final_process(sc_base_df,ready_to_target_dict)
dr_final_df = final_process(dr_base_df,ready_to_target_dict)

Lower uniqueness Better Performance, orgin uniqueness: 8206 , revised uniqueness : 7843
Lower uniqueness Better Performance, orgin uniqueness: 8206 , revised uniqueness : 7843
Lower uniqueness Better Performance, orgin uniqueness: 5630 , revised uniqueness : 5408
Lower uniqueness Better Performance, orgin uniqueness: 3564 , revised uniqueness : 3374
Lower uniqueness Better Performance, orgin uniqueness: 2210 , revised uniqueness : 2150


In [68]:
def make_counter_df_under_journal(base_df) :
    print('total length of this dataframe is {}'.format(len(base_df)))
    df1 = pd.DataFrame.from_dict(Counter(base_df['tuned_new_paper'].tolist()),orient='index',columns=['count'])
    df2 = df1.sort_values('count',ascending=False)
    return df2

In [69]:
display(Markdown('### 한국재무관리학회'))
display(make_counter_df_under_journal(fm_final_df).head(10))
display(Markdown('### 한국재무학회'))
display(make_counter_df_under_journal(f_final_df).head(10))
display(Markdown('### 한국증권학회'))
display(make_counter_df_under_journal(sc_final_df).head(10))
display(Markdown('### 한국파생상품학회'))
display(make_counter_df_under_journal(dr_final_df).head(10))

### 한국재무관리학회

total length of this dataframe is 10896


,count
theory of the firm: managerial behavior agency costs and ownership structure////1976,51
limit order trading////1996,44
‘////nan,44
results////2007,43
agency costs of free cash flow : corporate finance and takeovers////1986,38
a simple positive semi-definite heteroscedasticity and autocorrelation consistent covariance matrix////1987,31
common risk factors in the returns [provider:earticle] on stocks and bonds////1993,31
corporate financing and investment decisions when firms have information that investors do not have////1984,31
heteroscedasticity in stock returns////1990,25
the cross-section of expected stock returns////1992,23


### 한국재무학회

total length of this dataframe is 7045


...

total length of this dataframe is 7331


,count
common risk factors in the returns [provider:earticle] on stocks and bonds////1993,203
theory of the firm: managerial behavior agency costs and ownership structure////1976,123
on persistence in mutual fund performance////1997,102
risk return and equilibrium : empirical tests////1973,73
the pricing of options and corporate liabilities////1973,73
returns to buying winners and selling losers: implications for stock market efficiency////1993,67
tunneling or value added? evidence from mergers by korean business grups////2002,58
results////2007,52
corporate financing and investment decisions when firms have information that investors do not have////1984,45
agency costs of free cash flow : corporate finance and takeovers////1986,43


### 한국파생상품학회

total length of this dataframe is 3178


,count
‘////nan,139
common risk factors in the returns [provider:earticle] on stocks and bonds////1993,41
stock index futures////2006,33
ratio////2007,22
studies of stock price volatility changes////1976,21
results////2007,19
asset pricing////2001,17
a theory of volatility spreads////2006,16
explaining credit default swap spreads with the equity volatility and jump risks of individual firms////2009,15
stock return characteristics skew laws and the differential pricing of individual equity options////2003,15
